In [1]:
import numpy as np
import pandas as pd

from warnings import filterwarnings
filterwarnings(action='ignore', category=FutureWarning)

COKE = '/kaggle/input/coca-cola-stock-live-and-updated/Coca-Cola_stock_history.csv'
LONG = 200
SHORT = 50
USECOLS = ['Date', 'Close']

df = pd.read_csv(filepath_or_buffer=COKE, usecols=USECOLS)
# we have dates in two formats, with and without time; we need to clean them
df['date'] = pd.to_datetime(df['Date'].str.split().str[0])

# now we can drop all the columns we won't use
df = df.drop(columns=['Date'])

df.head()

,Close,date
0,0.050016,1962-01-02
1,0.048902,1962-01-03
2,0.049273,1962-01-04
3,0.048159,1962-01-05
4,0.047664,1962-01-08


In [2]:
from plotly.express import line
for log_y in [False, True]:
    line(data_frame=df, x='date', y='Close', log_y=log_y, ).show()

Let's look at our indicators for just the New Coke era. 

In [3]:
from datetime import datetime

new_df = df[df['date'] > pd.Timestamp('1985-04-23')].copy().reset_index(drop=True)

# Buy and hold total and annualized return

If we buy and hold what is our total return?

In [4]:
# what are the starting and ending values and the difference between them?
new_df['Close'].values[0], new_df['Close'].values[-1], new_df['Close'].values[-1] - new_df['Close'].values[0] 

(0.58621794, 59.38999939, 58.80378145)

In [5]:
total_return = new_df['Close'].values[-1]/new_df['Close'].values[0]
print('total return: {:5.2f} pct'.format(100 * total_return))
years = (new_df['date'].max() - new_df['date'].min()).days / 365.25
print('years: {:5.2f}'.format(years))

total return: 10131.04 pct
years: 37.51


In [6]:
from math import log
from math import exp

def get_rate(start_value: float, end_value: float, years: float) -> float:
    rate = exp(1.0/years * log(end_value/start_value)) - 1.0
    return rate

annualized_rate = get_rate(start_value=new_df['Close'].values[0], end_value=new_df['Close'].values[-1], years=years)

print('annualized pct rate: {:6.3f}'.format(100 * annualized_rate))

annualized pct rate: 13.103


A 13% annualized return is nothing to sneeze at.

In [7]:
# does our annualized percentage reproduce the total return?
print('total return based on annualized return: {:5.2f}'.format(1.13103 ** 37.51))
print('approximation difference: {:5.2f}'.format(total_return - 1.13103 ** 37.51))

total return based on annualized return: 101.35
approximation difference: -0.04


# Simple moving average trading

We want to trade the difference between the short-term moving average and the long-term moving average; when the short-term moving average is higher we buy and when the long-term moving average is higher we sell. We also want to compare how much money we make to what we would have made if we bought and held, and we want to monitor the number of trades, as transactions generally cost money and eat into our proceeds; we assume for simplicity that we trade in sufficient volume that the transaction cost is insignificant compared to our trade size in dollars.

In [8]:
from numpy import nan
from numpy import where
from plotly.express import line
from plotly.express import scatter

# now we need to add our trade signal components
new_df['short'] = new_df['Close'].rolling(window=SHORT).mean()
new_df['long'] = new_df['Close'].rolling(window=LONG).mean()

line(data_frame=new_df, x='date', y=['short', 'long'], log_y=True).show()

new_df['signal'] = where(new_df['short'] > new_df['long'], 1, 0)
new_df['position'] = new_df['signal'].diff().fillna(value=1) # buy on day one
new_df['buy'] = where(new_df['position'] == 1, new_df['Close'], nan)
new_df['sell'] = where(new_df['position'] == -1, new_df['Close'], nan)
new_df['profit'] = (new_df['sell'].fillna(value=0) - new_df['buy'].fillna(value=0)).cumsum()

scatter(data_frame=new_df, x='date', y=['buy', 'sell'], log_y=True).show()

print('proceeds: {:5.2f}'.format(new_df['sell'].sum() - new_df['buy'].sum()))
print('pct of buy and hold: {:5.2f}'.format(100 * (new_df['sell'].sum() - new_df['buy'].sum())/(new_df['Close'].values[-1] - new_df['Close'].values[0])))
print('we made trades on {} days out of {} trading days.'.format((new_df['position'] != 0).sum(), len(new_df)))

proceeds: 26.59
pct of buy and hold: 45.22
we made trades on 51 days out of 9455 trading days.


Of course what constitutes a short term and a long term as far as moving averages are concerned is partly a matter of choice when we have this much data, so let's parameterize our profit (or loss) by the two window sizes.

In [9]:
def strategy(input_df: pd.DataFrame, quote: str, long: int, short: int) -> float:
    work_df = input_df.copy()
    work_df['short'] = work_df[quote].rolling(window=short).mean()
    work_df['long'] = work_df[quote].rolling(window=long).mean()
    work_df['signal'] = where(work_df['short'] > work_df['long'], 1, 0)
    work_df['position'] = work_df['signal'].diff().fillna(value=1) # buy on day one
    work_df['buy'] = where(work_df['position'] == 1, work_df[quote], nan)
    work_df['sell'] = where(work_df['position'] == -1, work_df[quote], nan)
    return work_df['sell'].sum() - work_df['buy'].sum(), (work_df['position'] != 0).sum()

short = []
long = []
profit = []
trades = []
for short_value in range(4, 124, 4):
    for long_value in range(8, 252, 4):
        short.append(short_value)
        long.append(long_value)
        value, count = strategy(input_df = new_df[['Close']], quote='Close', long=long_value, short=short_value,)
        profit.append(value)
        trades.append(count)
size = [abs(item) for item in profit]
plot_df = pd.DataFrame(data={'short': short, 'long': long, 'profit/loss': profit, 'size': size, 'trades': trades})
scatter(x='short', y='long', color='profit/loss', size='size', height=900, data_frame=plot_df, hover_data=['trades']).show()


Is it surprising that our largest profits and largest losses are in neighboring parts of the parameter space?
Or that some of our largest returns are in a part of the space where our window sizes are reversed, where our 'long' window is smaller than our 'short' window?

Maybe if we want to pick a trading strategy we want to look not for the points that produce the highest return, but maybe a region in the parameter space that produces a good return on average, as this will give us some reason to believe that our profit won't be volatile as a function of the window sizes. Let's look at the same data in a heatmap.

In [10]:
from plotly.express import density_heatmap
density_heatmap(data_frame=plot_df, x='short', z='profit/loss', y='long',  height=900, nbinsx=30, nbinsy=30, histfunc='avg')

It turns out we have multiple regions in the parameter space that produce more than 25 dollars of profit (per unit). Unfortunately the only case that is better than buy and hold is one we would probably never try: the long/short windows are reversed, the trade count is really high, and some of the neighboring cases are losses.

In [11]:
from plotly.express import histogram
histogram(data_frame=plot_df, x='profit/loss', ).show()
histogram(data_frame=plot_df, x='trades', ).show()

Let's do the histograms above but only for cases where our short window is smaller than our long window.

In [12]:
short_ = []
long_ = []
profit_ = []
trades_ = []
for short_value in range(4, 124, 4):
    for long_value in range(short_value + 4, 252, 4):
        short_.append(short_value)
        long_.append(long_value)
        value, count = strategy(input_df = new_df[['Close']], quote='Close', long=long_value, short=short_value,)
        profit_.append(value)
        trades_.append(count)
plot2_df = pd.DataFrame(data={'short': short_, 'long': long_, 'profit/loss': profit_, 'size': [abs(item) for item in profit_], 'trades': trades_})
scatter(x='short', y='long', color='profit/loss', size='size', height=900, data_frame=plot2_df, hover_data=['trades']).show()


Now we can clearly see that we aren't doing any reversed cases.

In [13]:
histogram(data_frame=plot2_df, x='profit/loss')

In this case the profit mode of our profit/loss distribution is much larger.

In [14]:
histogram(data_frame=plot2_df, x='trades')